## Loading data

In [1]:
import pandas as pd
import numpy as np
import math
import qexpy as q
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
from darts import TimeSeries
from darts.dataprocessing.transformers import MissingValuesFiller
from darts.dataprocessing.transformers import Scaler
from darts.models import TCNModel


/opt/anaconda3/envs/WDN_sim/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('../output/synthetic_water_data_minute_1000.csv')
df.head()

/var/folders/7y/c70b2kjs6bdfsmfnvfrmypnh0000gn/T/ipykernel_22065/3265498077.py:1: DtypeWarning: Columns (17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../output/synthetic_water_data_minute_1000.csv')


,timestamp,house_id,velocity_m_per_s,flow_m3_s,flow_gpm,upstream_transit_time_s,downstream_transit_time_s,delta_t_ns,pressure_psi,pipe_material,...,l_path_m,pipe_burst_leak,leak_type,leak_category,leak_branch,leak_pipe,c_est_m_per_s,temp_est_c,n_traverses,theta_deg
0,2025-01-01T00:00:00,1,2.4,0.00476,75.447525,0.000059,0.000059,-96.761394,-95.084198,Copper,...,0.087572,False,none,NaN,NaN,NaN,1473.8,17.348762,4,60
1,2025-01-01T00:15:00,1,2.4,0.00476,75.447525,0.000059,0.000059,-96.761394,-95.084198,Copper,...,0.087572,False,none,NaN,NaN,NaN,1473.8,17.348762,4,60
2,2025-01-01T00:30:00,1,2.4,0.00476,75.447525,0.000059,0.000059,-96.761394,-95.084198,Copper,...,0.087572,False,none,NaN,NaN,NaN,1473.8,17.348762,4,60
3,2025-01-01T00:45:00,1,2.4,0.00476,75.447525,0.000059,0.000059,-96.761394,-95.084198,Copper,...,0.087572,False,none,NaN,NaN,NaN,1473.8,17.348762,4,60
4,2025-01-01T01:00:00,1,2.4,0.00476,75.447525,0.000059,0.000059,-96.761394,-95.084198,Copper,...,0.087572,False,none,NaN,NaN,NaN,1473.8,17.348762,4,60


In [3]:
df.describe()

,house_id,velocity_m_per_s,flow_m3_s,flow_gpm,upstream_transit_time_s,downstream_transit_time_s,delta_t_ns,pressure_psi,pipe_width_in,od_mm,wall_mm,id_mm,l_path_m,c_est_m_per_s,temp_est_c,n_traverses,theta_deg
count,3.505200e+07,3.505200e+07,3.505200e+07,3.505200e+07,3.505200e+07,3.505200e+07,3.505200e+07,3.505200e+07,3.505200e+07,3.505200e+07,3.505200e+07,3.505200e+07,3.505200e+07,3.505200e+07,3.505200e+07,35052000.0,35052000.0
mean,5.005000e+02,2.689800e+00,9.935196e-03,1.574760e+02,6.968063e-05,6.955361e-05,-1.270280e+02,-3.654508e+02,8.807500e-01,2.237105e+01,7.857300e-02,2.221390e+01,1.026016e-01,1.473800e+03,1.734876e+01,4.0,60.0
std,2.886750e+02,2.998266e-01,6.895592e-03,1.092973e+02,9.899222e-06,9.881395e-06,2.285495e+01,7.120368e+02,1.248677e-01,3.171639e+00,3.353200e-02,3.155866e+00,1.457632e-02,9.920878e-06,2.984665e-06,0.0,0.0
min,1.000000e+00,2.400000e+00,3.472000e-03,5.503231e+01,5.915379e-05,5.903350e-05,-1.604421e+02,-9.397137e+03,7.500000e-01,1.905000e+01,4.500000e-02,1.885600e+01,8.709213e-02,1.473800e+03,1.734876e+01,4.0,60.0
25%,2.507500e+02,2.400000e+00,4.690000e-03,7.433801e+01,5.946794e-05,5.937118e-05,-1.604421e+02,-3.506985e+02,7.500000e-01,1.905000e+01,5.000000e-02,1.896000e+01,8.757249e-02,1.473800e+03,1.734876e+01,4.0,60.0
50%,5.005000e+02,2.400000e+00,5.614000e-03,8.898370e+01,7.889891e-05,7.873847e-05,-1.291173e+02,-8.592019e+01,1.000000e+00,2.540000e+01,5.000000e-02,2.515000e+01,1.161629e-01,1.473800e+03,1.734876e+01,4.0,60.0
75%,7.502500e+02,3.000000e+00,1.604400e-02,2.543025e+02,7.935332e-05,7.922420e-05,-1.202898e+02,1.688394e+01,1.000000e+00,2.540000e+01,1.250000e-01,2.530000e+01,1.168557e-01,1.473800e+03,1.734877e+01,4.0,60.0
max,1.000000e+03,3.000000e+00,4.391800e-02,6.961144e+02,7.935332e-05,7.922420e-05,-9.676139e+01,4.539687e+01,1.000000e+00,2.540000e+01,1.250000e-01,2.530000e+01,1.168557e-01,1.473800e+03,1.734877e+01,4.0,60.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35052000 entries, 0 to 35051999
Data columns (total 24 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   timestamp                  object 
 1   house_id                   int64  
 2   velocity_m_per_s           float64
 3   flow_m3_s                  float64
 4   flow_gpm                   float64
 5   upstream_transit_time_s    float64
 6   downstream_transit_time_s  float64
 7   delta_t_ns                 float64
 8   pressure_psi               float64
 9   pipe_material              object 
 10  pipe_width_in              float64
 11  od_mm                      float64
 12  wall_mm                    float64
 13  id_mm                      float64
 14  l_path_m                   float64
 15  pipe_burst_leak            bool   
 16  leak_type                  object 
 17  leak_category              object 
 18  leak_branch                object 
 19  leak_pipe                  object 
 20  

In [5]:
df.isnull().sum()

timestamp                           0
house_id                            0
velocity_m_per_s                    0
flow_m3_s                           0
flow_gpm                            0
upstream_transit_time_s             0
downstream_transit_time_s           0
delta_t_ns                          0
pressure_psi                        0
pipe_material                       0
pipe_width_in                       0
od_mm                               0
wall_mm                             0
id_mm                               0
l_path_m                            0
pipe_burst_leak                     0
leak_type                           0
leak_category                30946137
leak_branch                  30946137
leak_pipe                    30946137
c_est_m_per_s                       0
temp_est_c                          0
n_traverses                         0
theta_deg                           0
dtype: int64

In [6]:
nan_cols = ['leak_category','leak_branch','leak_pipe']
df[nan_cols] = df[nan_cols].fillna(value='none')
df.head()

,timestamp,house_id,velocity_m_per_s,flow_m3_s,flow_gpm,upstream_transit_time_s,downstream_transit_time_s,delta_t_ns,pressure_psi,pipe_material,...,l_path_m,pipe_burst_leak,leak_type,leak_category,leak_branch,leak_pipe,c_est_m_per_s,temp_est_c,n_traverses,theta_deg
0,2025-01-01T00:00:00,1,2.4,0.00476,75.447525,0.000059,0.000059,-96.761394,-95.084198,Copper,...,0.087572,False,none,none,none,none,1473.8,17.348762,4,60
1,2025-01-01T00:15:00,1,2.4,0.00476,75.447525,0.000059,0.000059,-96.761394,-95.084198,Copper,...,0.087572,False,none,none,none,none,1473.8,17.348762,4,60
2,2025-01-01T00:30:00,1,2.4,0.00476,75.447525,0.000059,0.000059,-96.761394,-95.084198,Copper,...,0.087572,False,none,none,none,none,1473.8,17.348762,4,60
3,2025-01-01T00:45:00,1,2.4,0.00476,75.447525,0.000059,0.000059,-96.761394,-95.084198,Copper,...,0.087572,False,none,none,none,none,1473.8,17.348762,4,60
4,2025-01-01T01:00:00,1,2.4,0.00476,75.447525,0.000059,0.000059,-96.761394,-95.084198,Copper,...,0.087572,False,none,none,none,none,1473.8,17.348762,4,60


In [7]:
df.isnull().sum()

timestamp                    0
house_id                     0
velocity_m_per_s             0
flow_m3_s                    0
flow_gpm                     0
upstream_transit_time_s      0
downstream_transit_time_s    0
delta_t_ns                   0
pressure_psi                 0
pipe_material                0
pipe_width_in                0
od_mm                        0
wall_mm                      0
id_mm                        0
l_path_m                     0
pipe_burst_leak              0
leak_type                    0
leak_category                0
leak_branch                  0
leak_pipe                    0
c_est_m_per_s                0
temp_est_c                   0
n_traverses                  0
theta_deg                    0
dtype: int64

## Calculating velocity and flow erros

In [ ]:
def calculate_errors(df):
    theta_deg = 60
    theta_cos = np.cos(np.radians(theta_deg))
    epsilon = 1e-10

    # Extract needed fields as arrays
    L = df['l_path_m'].values
    t_u = df['upstream_transit_time_s'].values
    t_d = df['downstream_transit_time_s'].values
    V_actual = df['velocity_m_per_s'].values
    Q_actual = df['flow_m3_s'].values
    id_m = df['id_mm'].values / 1000  # mm to m

    # Step 1: Estimate velocity
    V_est = (L / (2 * theta_cos)) * ((1 / t_d) - (1 / t_u))

    # Step 2: Area of circular pipe
    A = (np.pi/4) * (id_m ** 2)

    # Step 3: Estimate flow
    Q_est = V_est * A

    # Step 4: Error metrics
    velocity_error = np.abs(V_actual - V_est) / np.maximum(np.abs(V_actual), epsilon)
    flow_rate_error = np.abs(Q_actual - Q_est) / np.maximum(np.abs(Q_actual), epsilon)

    # Step 5: Add to DataFrame
    df['V_est'] = V_est
    df['Q_est'] = Q_est
    df['velocity_error'] = velocity_error
    df['flow_rate_error'] = flow_rate_error
    df['velocity_error_pass'] = velocity_error <= 0.05
    df['flow_rate_error_pass'] = flow_rate_error <= 0.05

    return df

In [9]:
# Apply error calculations to the dataframe
df = calculate_errors(df)

df

,timestamp,house_id,velocity_m_per_s,flow_m3_s,flow_gpm,upstream_transit_time_s,downstream_transit_time_s,delta_t_ns,pressure_psi,pipe_material,...,c_est_m_per_s,temp_est_c,n_traverses,theta_deg,v_est,q_est,velocity_error,flow_rate_error,velocity_error_pass,flow_rate_error_pass
0,2025-01-01T00:00:00,1,2.4,0.004760,75.447525,0.000059,0.000059,-96.761394,-95.084198,Copper,...,1473.8,17.348762,4,60,2.4,0.000678,6.772360e-14,0.857646,True,False
1,2025-01-01T00:15:00,1,2.4,0.004760,75.447525,0.000059,0.000059,-96.761394,-95.084198,Copper,...,1473.8,17.348762,4,60,2.4,0.000678,6.772360e-14,0.857646,True,False
2,2025-01-01T00:30:00,1,2.4,0.004760,75.447525,0.000059,0.000059,-96.761394,-95.084198,Copper,...,1473.8,17.348762,4,60,2.4,0.000678,6.772360e-14,0.857646,True,False
3,2025-01-01T00:45:00,1,2.4,0.004760,75.447525,0.000059,0.000059,-96.761394,-95.084198,Copper,...,1473.8,17.348762,4,60,2.4,0.000678,6.772360e-14,0.857646,True,False
4,2025-01-01T01:00:00,1,2.4,0.004760,75.447525,0.000059,0.000059,-96.761394,-95.084198,Copper,...,1473.8,17.348762,4,60,2.4,0.000678,6.772360e-14,0.857646,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35051995,2025-12-31T23:00:00,1000,2.4,0.003962,62.798973,0.000059,0.000059,-96.761394,-50.402031,Copper,...,1473.8,17.348762,4,60,2.4,0.000678,6.772360e-14,0.828974,True,False
35051996,2025-12-31T23:15:00,1000,2.4,0.003962,62.798973,0.000059,0.000059,-96.761394,-50.402031,Copper,...,1473.8,17.348762,4,60,2.4,0.000678,6.772360e-14,0.828974,True,False
35051997,2025-12-31T23:30:00,1000,2.4,0.003962,62.798973,0.000059,0.000059,-96.761394,-50.402031,Copper,...,1473.8,17.348762,4,60,2.4,0.000678,6.772360e-14,0.828974,True,False
35051998,2025-12-31T23:45:00,1000,2.4,0.003962,62.798973,0.000059,0.000059,-96.761394,-50.402031,Copper,...,1473.8,17.348762,4,60,2.4,0.000678,6.772360e-14,0.828974,True,False


## Leak df

In [11]:
df.columns.tolist()

['timestamp',
 'house_id',
 'velocity_m_per_s',
 'flow_m3_s',
 'flow_gpm',
 'upstream_transit_time_s',
 'downstream_transit_time_s',
 'delta_t_ns',
 'pressure_psi',
 'pipe_material',
 'pipe_width_in',
 'od_mm',
 'wall_mm',
 'id_mm',
 'l_path_m',
 'pipe_burst_leak',
 'leak_type',
 'leak_category',
 'leak_branch',
 'leak_pipe',
 'c_est_m_per_s',
 'temp_est_c',
 'n_traverses',
 'theta_deg',
 'v_est',
 'q_est',
 'velocity_error',
 'flow_rate_error',
 'velocity_error_pass',
 'flow_rate_error_pass']

In [13]:
leak_df = df.drop(['pressure_psi', ], axis=1)
leak_df.head()

,timestamp,house_id,velocity_m_per_s,flow_m3_s,flow_gpm,upstream_transit_time_s,downstream_transit_time_s,delta_t_ns,pipe_material,pipe_width_in,...,c_est_m_per_s,temp_est_c,n_traverses,theta_deg,v_est,q_est,velocity_error,flow_rate_error,velocity_error_pass,flow_rate_error_pass
0,2025-01-01T00:00:00,1,2.4,0.00476,75.447525,0.000059,0.000059,-96.761394,Copper,0.75,...,1473.8,17.348762,4,60,2.4,0.000678,6.772360e-14,0.857646,True,False
1,2025-01-01T00:15:00,1,2.4,0.00476,75.447525,0.000059,0.000059,-96.761394,Copper,0.75,...,1473.8,17.348762,4,60,2.4,0.000678,6.772360e-14,0.857646,True,False
2,2025-01-01T00:30:00,1,2.4,0.00476,75.447525,0.000059,0.000059,-96.761394,Copper,0.75,...,1473.8,17.348762,4,60,2.4,0.000678,6.772360e-14,0.857646,True,False
3,2025-01-01T00:45:00,1,2.4,0.00476,75.447525,0.000059,0.000059,-96.761394,Copper,0.75,...,1473.8,17.348762,4,60,2.4,0.000678,6.772360e-14,0.857646,True,False
4,2025-01-01T01:00:00,1,2.4,0.00476,75.447525,0.000059,0.000059,-96.761394,Copper,0.75,...,1473.8,17.348762,4,60,2.4,0.000678,6.772360e-14,0.857646,True,False


In [14]:
leak_df.query('pipe_burst_leak == 1').head(20)

,timestamp,house_id,velocity_m_per_s,flow_m3_s,flow_gpm,upstream_transit_time_s,downstream_transit_time_s,delta_t_ns,pipe_material,pipe_width_in,...,c_est_m_per_s,temp_est_c,n_traverses,theta_deg,v_est,q_est,velocity_error,flow_rate_error,velocity_error_pass,flow_rate_error_pass
4104,2025-01-12T17:45:00,2,2.4,0.017052,270.279663,0.000059,0.000059,-96.761394,Copper,0.75,...,1473.8,17.348762,4,60,2.4,0.000678,6.772360e-14,0.960262,True,False
4105,2025-01-12T18:00:00,2,2.4,0.017052,270.279663,0.000059,0.000059,-96.761394,Copper,0.75,...,1473.8,17.348762,4,60,2.4,0.000678,6.772360e-14,0.960262,True,False
4106,2025-01-12T18:15:00,2,2.4,0.017052,270.279663,0.000059,0.000059,-96.761394,Copper,0.75,...,1473.8,17.348762,4,60,2.4,0.000678,6.772360e-14,0.960262,True,False
4107,2025-01-12T18:30:00,2,2.4,0.017052,270.279663,0.000059,0.000059,-96.761394,Copper,0.75,...,1473.8,17.348762,4,60,2.4,0.000678,6.772360e-14,0.960262,True,False
4108,2025-01-12T18:45:00,2,2.4,0.017052,270.279663,0.000059,0.000059,-96.761394,Copper,0.75,...,1473.8,17.348762,4,60,2.4,0.000678,6.772360e-14,0.960262,True,False
4109,2025-01-12T19:00:00,2,2.4,0.017052,270.279663,0.000059,0.000059,-96.761394,Copper,0.75,...,1473.8,17.348762,4,60,2.4,0.000678,6.772360e-14,0.960262,True,False
4110,2025-01-12T19:15:00,2,2.4,0.017052,270.279663,0.000059,0.000059,-96.761394,Copper,0.75,...,1473.8,17.348762,4,60,2.4,0.000678,6.772360e-14,0.960262,True,False
4111,2025-01-12T19:30:00,2,2.4,0.017052,270.279663,0.000059,0.000059,-96.761394,Copper,0.75,...,1473.8,17.348762,4,60,2.4,0.000678,6.772360e-14,0.960262,True,False
4112,2025-01-12T19:45:00,2,2.4,0.017052,270.279663,0.000059,0.000059,-96.761394,Copper,0.75,...,1473.8,17.348762,4,60,2.4,0.000678,6.772360e-14,0.960262,True,False
4113,2025-01-12T20:00:00,2,2.4,0.017052,270.279663,0.000059,0.000059,-96.761394,Copper,0.75,...,1473.8,17.348762,4,60,2.4,0.000678,6.772360e-14,0.960262,True,False


## Calculating acousting path of ultrasonic waves

Formula to calculate the acoustic path (L) of the ultrasonic waves for a given pipe length:
$$
L = \frac{N}{\sin\theta}
$$

$
L = \mathrm{acoustic\ path}\;(m) \\
N = \mathrm{wave\ reflections + 1}\;(const) \\
d = \mathrm{inner\ pipe\ diameter}\;(m) \\
\theta = \mathrm{angle\ of\ incidence}\;(radians)
$

In [16]:
def inner_diameter(d):
    

IndentationError: expected an indented block (1178739250.py, line 2)

In [17]:
def acoustic_path(d):
    theta = math.radians(60) # Angle of incidence for 0.75 and 1.0 inch pipes is roughly 60 degrees
    d_mm = d * 0.0254 # inches to m
    N = 4 # represents the W path ultasonic waves take in 0.75 to 1.0 inch pipes
    return N * d_mm / math.cos(theta)


def acoustic_path_df(df):
    df['acoustic_path'] = acoustic_path(df['pipe_width_in'])
    return df

In [18]:
acoustic_path_df(leak_df)

,timestamp,house_id,velocity_m_per_s,flow_m3_s,flow_gpm,upstream_transit_time_s,downstream_transit_time_s,delta_t_ns,pipe_material,pipe_width_in,...,temp_est_c,n_traverses,theta_deg,v_est,q_est,velocity_error,flow_rate_error,velocity_error_pass,flow_rate_error_pass,acoustic_path
0,2025-01-01T00:00:00,1,2.4,0.004760,75.447525,0.000059,0.000059,-96.761394,Copper,0.75,...,17.348762,4,60,2.4,0.000678,6.772360e-14,0.857646,True,False,0.1524
1,2025-01-01T00:15:00,1,2.4,0.004760,75.447525,0.000059,0.000059,-96.761394,Copper,0.75,...,17.348762,4,60,2.4,0.000678,6.772360e-14,0.857646,True,False,0.1524
2,2025-01-01T00:30:00,1,2.4,0.004760,75.447525,0.000059,0.000059,-96.761394,Copper,0.75,...,17.348762,4,60,2.4,0.000678,6.772360e-14,0.857646,True,False,0.1524
3,2025-01-01T00:45:00,1,2.4,0.004760,75.447525,0.000059,0.000059,-96.761394,Copper,0.75,...,17.348762,4,60,2.4,0.000678,6.772360e-14,0.857646,True,False,0.1524
4,2025-01-01T01:00:00,1,2.4,0.004760,75.447525,0.000059,0.000059,-96.761394,Copper,0.75,...,17.348762,4,60,2.4,0.000678,6.772360e-14,0.857646,True,False,0.1524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35051995,2025-12-31T23:00:00,1000,2.4,0.003962,62.798973,0.000059,0.000059,-96.761394,Copper,0.75,...,17.348762,4,60,2.4,0.000678,6.772360e-14,0.828974,True,False,0.1524
35051996,2025-12-31T23:15:00,1000,2.4,0.003962,62.798973,0.000059,0.000059,-96.761394,Copper,0.75,...,17.348762,4,60,2.4,0.000678,6.772360e-14,0.828974,True,False,0.1524
35051997,2025-12-31T23:30:00,1000,2.4,0.003962,62.798973,0.000059,0.000059,-96.761394,Copper,0.75,...,17.348762,4,60,2.4,0.000678,6.772360e-14,0.828974,True,False,0.1524
35051998,2025-12-31T23:45:00,1000,2.4,0.003962,62.798973,0.000059,0.000059,-96.761394,Copper,0.75,...,17.348762,4,60,2.4,0.000678,6.772360e-14,0.828974,True,False,0.1524


In [19]:
def calc_sound_speed(t_up: float, t_down: float, L: float) -> float:
    """
    Speed of sound in the water column (m s-1).

    Parameters
    ----------
    t_up   : upstream transit time, s  (pulse sent *against* the flow)
    t_down : downstream transit time, s  (pulse sent *with* the flow)
    L      : acoustic path length between the two transducers, m

    Returns
    -------
    c_ms   : speed of sound in the fluid, m s-1
    """
    return L / (0.5 * (t_up + t_down))


def calc_flow_velocity(t_up: float, t_down: float, L: float) -> float:
    """
    Average axial flow velocity in the pipe (m s-1).

    v = (L / 2) · (1/t_up − 1/t_down)

    Parameters
    ----------
    t_up, t_down, L : see calc_sound_speed

    Returns
    -------
    v_ms : bulk flow velocity, m s-1
    """
    return (L / 2.0) * ((1.0 / t_up) + (1.0 / t_down))


def temperature_from_sound_speed(c_ms: float, tol: float = 1e-7, max_iter: int = 50) -> float:
    """
    Convert sound speed to temperature (°C) for fresh water at ≈1 atm
    by numerically inverting the UNESCO/ITS-90 polynomial (0–100 °C):

        c = Σ C_i · T^i ,    i = 0…5

    Coefficients for salinity = 0 ppt, pressure = 0 bar:               │
        C0 = 1402.388,  C1 = 5.03830,    C2 = −5.81090 E−2
        C3 = 3.3432 E−4, C4 = −1.47797 E−6, C5 = 3.1419 E−9  :contentReference[oaicite:1]{index=1}

    Newton iteration is used; the linear estimate (c−1400)/4.5 seeds it.

    Parameters
    ----------
    c_ms : measured sound speed, m s-1
    tol  : convergence tolerance on temperature, °C
    max_iter : maximum Newton iterations

    Returns
    -------
    T_C  : temperature, °C
    """

    # polynomial coefficients
    C0, C1, C2 = 1402.388, 5.03830, -5.81090e-2
    C3, C4, C5 = 3.3432e-4, -1.47797e-6, 3.1419e-9

    # initial guess (good to ±2 °C over 0-40 °C)
    T = (c_ms - 1400.0) / 4.5

    for _ in range(max_iter):
        f  = (C0 + C1*T + C2*T**2 + C3*T**3 + C4*T**4 + C5*T**5) - c_ms
        fp =  (C1 + 2*C2*T + 3*C3*T**2 + 4*C4*T**3 + 5*C5*T**4)
        T_new = T - f / fp
        if abs(T_new - T) < tol:
            return T_new
        T = T_new

    # fall-back (should rarely trigger)
    return T


def estimate_temperature(t_up: float, t_down: float, L: float) -> float:
    """
    End-to-end helper: feed it the two transit times and the path length,
    get back the water temperature (°C).

    Parameters
    ----------
    t_up, t_down, L : see calc_sound_speed

    Returns
    -------
    T_C : temperature, °C
    """
    c_ms = calc_sound_speed(t_up, t_down, L)
    return temperature_from_sound_speed(c_ms)


In [20]:
t_up   = 0.00003    # s
t_down = 0.00002    # s
L      = 0.0879     # m (50 mm acoustic path)

temp_C = estimate_temperature(t_up, t_down, L)
print(f"Water temperature ≈ {temp_C:0.2f} °C")
calc_flow_velocity(t_up, t_down, L)

Water temperature ≈ 329.92 °C


3662.5

In [21]:
t_up   = 0.00003    # s
t_down = 0.00002    # s
L      = 0.0879     # m (50 mm acoustic path)

c = L/2 * (1/t_up + 1/t_down)
print(c)
T =  0.04T

SyntaxError: invalid syntax (3717126241.py, line 7)

Ultrasonic constants at 19.3C

In [ ]:
v_19 = 1470 #m/s (+- 4)
f_19 = 1606732.976 # Hz
l_19 = 0.0009149 # m

delta = 2.5 # change in v due to 1 degree change in temperature 

Calcuate speed of ultrasonic waves given upstream and downstream times
Subtract the difference between the data point sound speed and the speed at 19.3C
Divide the difference by 2.5 to figiure out chagne in temperature
Add/subtract chnage in temperature to 19.3C to calulate final temperature of water

Check that upstream and downstream times make sense given flow speed

Use qexpy for magin of error calculations